In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
 cd /content/drive/MyDrive/GitHub/2020-2-OSSP1-WhatsUp-5/

/content/drive/MyDrive/GitHub/2020-2-OSSP1-WhatsUp-5


In [ ]:
!pip install -r requirements.txt

In [4]:
import os

import numpy as np
import tensorflow as tf

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize

import pandas as pd
import kss
import random

from utils.utils import get_jongsung_TF
from tqdm import tqdm

In [5]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [7]:
TOKENIZER_PATH = './model/gpt_ckpt/gpt2_kor_tokenizer.spiece'
tokenizer = SentencepieceTokenizer(TOKENIZER_PATH, num_best=0, alpha=0)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

## 텍스트 생성-코드

In [8]:
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])  
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])

In [9]:
# GPT Model을 활용한 텍스트 생성
def generate_sent(seed_word, model, max_step=400, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    toked = tokenizer(sent)
    for _ in tqdm(range(max_step)):
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':
            break
        #print("생성된 토큰 : "+gen)
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)
    return sent

In [10]:
# 불완전한 종결 문장 제거
def preprocessing(content):
    text = []
    article = ""
    for s in kss.split_sentences(content):
        text.append(s)
    if text[-1][-1] != '.':
        text = text[:-1]
    for t in text:
        article += t + " "
    return article

In [11]:
# DB에서 columns에 있는 열 중 랜덤한 정보를 가져온다.
def random_info(df, keyword, columns):
    column = random.choice(columns)
    for i in range(len(df)):
        if df.loc[i, 'disease'] == keyword:
            csv_text = df.loc[i, column]
            return csv_text 
    return ""

In [12]:
# 기사 생성
# return : title, content
def generate_article(keyword, title_path, content_path, df):
    title_model = GPT2Model(title_path)    # 제목 생성 모델 load
    title = generate_sent(keyword, title_model,top_k=0, top_p=0.90)     # 제목 생성 모델을 활용하여 기사 제목 생성 
    content_model = GPT2Model(content_path)     # 본문 생성 모델

    # 본문 생성 모델의 input 값으로 제목을 넣기 위한 전처리
    # 제목이 '.' 으로 끝나지 않을시 .을 추가한다.
    title = title.strip()
    if title[-1] not in ['.', '?', '!']:
        title += '.'
    # 사용할지 말지 임의로 정하기
    #if '...' in title:
    #    title = title.split('...')[-1]
    #    if title[-1] not in ['.', '?', '!']:
    #        title += '.'

    # 본문 생성모델의 input
    # 예) [기사 제목]. + [질병명] + 이란(란)
    input_text = title + keyword

    # 종성에 따라 조사 추가    
    if get_jongsung_TF(keyword) == "T":
        input_text += '이란'
    else:
        input_text += '란'

    content = generate_sent(input_text, content_model, top_k=40, top_p=0.90)    # 본문 생성 모델을 활용하여 기사 본문 생성
    content = content.replace(title, "").strip()
    content = preprocessing(content)        # 불완전한 종결문장 제거

    csv_text = ""   # DB로 부터 불러올 데이터
    columns = ['cause', 'symptom', 'diagnosis', 'treat']

    # 기사 본문에 질병의 증상, 진단, 치료, 원인을 다루는지 확인하고 없는 것만 columns에 추가
    if '증상' in content:
        columns.remove('symptom')
    if '진단' in content:
        columns.remove('diagnosis')
    if '치료' in content:
        columns.remove('treat')
    if '원인' in content:
        columns.remove('cause')

    if columns:     # columns의 원소들 중 하나를 랜덤하게 선택하여 질병 정보를 출력
        csv_text = random_info(df=df, keyword=keyword, columns = columns)

    content = content + csv_text    
    content = preprocessing(content)    # 불완전한 종결문장 제거, 공백 교정
    return title, content

# Generate Article

In [14]:
TITLE_MODEL_PATH = './model/checkpoint/maxlen20_loss0.36_acc0.56/'
CONTENT_MODEL_PATH = './model/checkpoint/psy_loss2.0_acc0.32/'
DB_PATH = "./data/samsung_hospital_db.csv"
KEYWORD_PATH = './text_mining/output/disease_keyword.csv'

In [19]:
df = pd.read_csv(DB_PATH)      # 질병 DB load

# textming 모델에서 LDA 분석을 통하여 추출된 keyword LOAD
input_keyword_df = pd.read_csv(KEYWORD_PATH)

# 추출된 keywords에서 랜덤하게 keyword(질병명) 선택 기사 선택
idx = random.randint(0, len(input_keyword_df)-1)    
keyword = input_keyword_df.loc[idx, 'disease']

print("Keyword:", keyword)

Keyword: 우울증


In [20]:
title, content = generate_article(keyword=keyword, title_path=TITLE_MODEL_PATH, content_path=CONTENT_MODEL_PATH, df=df)

100%|██████████| 400/400 [00:49<00:00,  8.08it/s]


# Article

In [21]:
title

'우울증, 불면증, 공황장애 증상등 신경정신질환- 조기발견과 치료 필요.'

In [22]:
content

'우울증이란 우울감, 무기력감, 불안 및 의욕상실 같은 증상을 보이는 정신질환이다. 하지만 그 양상이 다르게 나타난다. 특히 이런 우울증은 신경정신질환 중에서도 우울한 증상을 잘 표현하지 못하는 조기발견과 치료가 중요하다. 특히 전문가들은 조기발견의 중요성을 강조하며 환자의 증상에 맞는 적절한 치료를 할 수 있도록 해야 한다고 강조한다. 우울증의 조기발견 및 치료의 첫걸음은 조기진단이다. 특히 우울증은 다른 우울증과는 달리 원인을 알기 어렵고, 치료방법이 명확하지 않아 완치가 어려울 수 있다. 따라서 증상이 있다고 생각될 때는 병원을 찾아 조기에 진단하는 것이 바람직하다. 우울증 초기에는 신경과전문의의 진료로 치료하게 된다. 그 후 우울증의 전 단계인 ‘우울’ 단계까지 진행된 경우에도 전문의 진찰을 받도록 한다. 신경정신질환은 조기발견으로 치료가 가능하다. 특히 최근에는 약물치료, 비약물치료, 가족치료 등 약물치료와 정신치료로 치료가 가능하다. 하지만 약물과 비약물치료는 환자 본인의 의지만으로 되는 것이 아니라 전문의의 정확한 진단과 지도가 필요하다. 따라서 전문가들은 우울증이 의심되는 경우 병원을 찾으면 전문의에게 반드시 상담을 받는 것이 좋다. 또 증상이 호전되지 않을 때는 생활습관개선을 통해 우울증의 재발이 줄어들도록 한다. 우울증 예방을 위해서는 우선 자신의 몸 상태에 주의하면서 정기적인 신체검사와 함께 정기적인 건강검진을 받는 것이 좋다. 평소 꾸준한 운동과 식이요법 등으로 뇌에 공급되는 혈류량을 늘리거나 뇌신경 활성도를 높여서 우울감을 떨치고 기분을 좋게 만들어 정신신경질환을 예방하는 데 도움이 된다. 또한 우울증이 있을 경우 스트레스를 줄이려고 과식, 과음, 폭식을 피하고 지나친 우울한 기분상태를 만들지 않도록 한다. 우울증을 일으키는 유전적 요인은 발견되지 않지만, 잘못된 식습관이나 심리적 원인이 복합적으로 작용해서 나타나는 경우가 많다. 특히 비타민C, 아스파라긴산, 철분 같은 항산화 성분이 많이 들어있는 음식과 항산화제를 꾸준히 복용하는 것만